In [2]:
import cv2
import numpy as np

# Retrieving Each Axis Coordinates for all Points

In [3]:
def get_col(Pts,col):
    return [row[col] for row in Pts]

# Calculating Distance and Classifying Each Point To a Class

In [4]:
def distanceAndClassify(Mu, Pts):
    dist = []
    Mu = np.asarray(Mu)
    Pts = np.asarray(Pts)
    for k in range(len(Pts)):
        sub = []
        for i in range(len(Mu)):
            sub.append(np.sqrt(((Pts[k]-Mu[i])**2).sum()))             #Distance by Euclidean Distance
        dist.append(sub.index(min(sub)))                                     #Storing the only the index of the Minimum distance to a Class
    return dist

# Classifying the Points to the respective Centroid

In [5]:
def classifyPlot(Pts, C, k):
    cl = []
    for i in range(k):
        temp = []
        for j in range(len(Pts)):
            if C[j] == i:                                    #Checks if the Point 'C[j]' is in the corresponding Class 'i'
                temp.append(Pts[j])                 #If so, adds the Point to the set of Points in Class 'i'
        cl.append(temp) 
    return cl

# Recomputing Mu with the Points in each Class

In [6]:
def compMu(Cls, k): 
    Mu_temp= []
    Mu = []
    for i in range(k):
        Mu_temp= []
        for j in range(len(Cls[0][0])):
            #Taking the Mean for each Coordinate of a Class
            Mu_temp.append(np.around((np.asarray(get_col(Cls[i], j))).mean(),3))  
        Mu.append(Mu_temp)
    return Mu

# Reshaping the image to have the respective centroid color for each pixel in the image

In [7]:
def convert_img(Cls, Mu, img):
    k = 0
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            img[i][j] = Mu[Cls[k]]
            k += 1
    return img

# Stopping Criteria for K-Means

In [8]:
def error(Mu, Old_Mu, threshold):
    for i in range(len(Mu)):
        for j in range(len(Mu[0])):
            if (abs(Mu[i][j] - Old_Mu[i][j]) > threshold):
                return 1
    return 0

# Color Quantization - Converting an image into 'k' No. of Colors

In [17]:
def color_quantization(image, k, threshold):
    print ("Color Quantization with "+str(k)+" Colors")
    #Reading the image
    img = cv2.imread(image)
    
    #Converting all the points into a 2D List
    #i.e Reshape [m,n,3] to [m*n,3] List to access each pixel in the image
    Pts = img.reshape(img.shape[0]*img.shape[1],3).tolist()
    
    #Storing the Centroids (Colors) in Mu
    Old_Mu = []
    Mu = []
    
    #Randomly generating 'k' Centroids
    for i in range(k):
        Mu.append((np.random.randint(low=0, high=255, size=3)).tolist())
        Old_Mu.append((np.random.randint(low=0,high=1,size=3)).tolist())
    
    print("Initial Centroids (Mu) : " + str(Mu))
    count = 0
    #Stops iterating when the difference between the old and new Centroids are below the threshold
    while(error(Mu,Old_Mu, threshold)): 
        
        #Calculating distance of each point w.r.t each Centroid and Classifing the Points to the corresponding Centroid
        Cls = distanceAndClassify(Mu,Pts)
        
        #Storing all points in terms of the Centroid to which they are assigned
        Classification = classifyPlot(Pts,Cls, k)
        
        #Storing previous Mu values to compare
        Old_Mu = Mu
        
        #Recomputing the Centroid Means by taking mean of each class formed
        Mu = compMu(Classification, k)
        count += 1
        print("Mu after updation "+str(count)+" : " + str(Mu))
        
        #Converting the 2D set of Points to a 3D Set of Points so that we can display the image
        k_img = convert_img(Cls,Mu,img)
    cv2.imwrite("Color_Quantization_for_"+str(k)+"_colors.jpg",k_img)
    return k_img